# Behavior Clone Experiment: data processing
This model has two layers, max pooling and a dropout. We are still using one lap of data, where driver tries to stay in the middle of the road. Now we are doing data processing and augmentation:
* normalizing the data and mean centering
* data augmentation by flipping the image and steering measurements

In [1]:
# load data from csv
import csv
import cv2
import numpy as np

lines = []
with open('../../../../data/bc_one_lap/driving_log.csv') as csvfile:
    rd = csv.reader(csvfile)
    for line in rd:
        lines.append(line)
        
images = []
measurements = []
# update path 
for line in lines:
    source_path = line[0]
    # split using final token
    filename = source_path.split('/')[-1]
    current_path = '../../../../data/bc_one_lap/IMG/' + filename
    # use opencv to load the image
    image = cv2.imread(current_path)
    # append to list of images + data augment
    images.append(image)
    images.append(cv2.flip(image, 1))
    # this is the steering measurement
    measurement = float(line[3])
    measurements.append(measurement)
    measurements.append(measurement*-1.0)
    
# now convert to numpy arrays for keras
X_train = np.array(images)
y_train = np.array(measurements)

In [6]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Convolution2D, Dropout, MaxPooling2D, Lambda
input_shape=(160,320,3)

# most NN model with 2 layers and dropout
model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Convolution2D(32,3,3,activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 158, 318, 32)  896         lambda_1[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 79, 159, 32)   0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 77, 157, 64)   18496       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [7]:
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2,
          shuffle=True, nb_epoch=3)

Train on 2377 samples, validate on 595 samples
Epoch 1/3
2377/2377 [==============================] - 135s - loss: 2.2312 - val_loss: 0.0038
Epoch 2/3
2377/2377 [==============================] - 142s - loss: 0.0195 - val_loss: 0.0038
Epoch 3/3
2377/2377 [==============================] - 142s - loss: 0.0195 - val_loss: 0.0038


In [8]:
model.save('../model.h5')

## Results
When I used this trained model the results were better. The car is negotiating the first curve, but once it reaches a sharper curve it goes off the road. This can be seen in the video below.

In [9]:
%%HTML
<video width="320" height="240" controls>
  <source src="basic_dataprocess.mp4" type="video/mp4">
</video>